# Level 1+2 Jupyter Notebook

To run this Jupyter Note book
1) Go to kernel → Restart & Run All
2) Scoll back to top of the page and hit here where it says "Click here to show/hide codes in this notebook" (optional)
3) Jupyer Notebook will run 

In [ ]:
from IPython.display import HTML
HTML('''
<script>
code_show=false;
function code_toggle(){
    if(code_show){$('.prompt, .input, .output_stderr, .output_error').hide();}
    else{$('.input, .prompt, .output_stderr, .output_error').show();}
    code_show=!code_show;
}
function initialize(){
    document.body.style.fontFamily='Palatino';
    var output=$('.output_subarea.output_text.output_stream.output_stdout');
    $.merge(output,$('.output_subarea.output_text.output_result'));
    for(var i=0;i<output.length;i++)for(var j=0;j<output[i].children.length;j++)
        output[i].children[j].style.fontFamily='Palatino';
    code_toggle();
}
$(document).ready(initialize);
</script>
Click <a href="javascript:code_toggle()">here</a> to show/hide codes in this notebook.
''')

In [ ]:
print("Code is running")

# Setup 
Be prepared to answer questions: 
    
    1) Type of Computer System 
    2) Test Type 
    3) Number of tests run  

In [ ]:
%%capture
!pip install openpyxl;
!pip install uncertainties;
!pip install tabulate;
!pip install xlrd; 
!pip install scipy; 
!pip install easygui; 

In [ ]:
import sys; sys.path.insert(0, '.')
import os
import pandas as pd
import uncertainties
from uncertainties import ufloat
import csv
from datetime import datetime as dt
from tabulate import tabulate
from LEMS_MakeInputFile_EnergyCalcs import LEMS_MakeInputFile_EnergyCalcs
import colorama
from colorama import Fore, Back, Style

In [ ]:
print("Please specify computer system. Put L for Linux and W for windows")
comp_program = str(input())

In [ ]:
if comp_program =='L' or comp_program == 'l':
    print("You have a Linux System")
    print("Please specify test type. Put 1894 for iso 1894 and 1952 for iso 1952")
    test_type = int(input())
elif comp_program == 'W' or comp_program == 'w': 
    print("You have a Windows System")
    print("Please specify test type. Put 1894 for iso 1894 and 1952 for iso 1952")
    test_type = int(input())
else: 
    print("You have not entered a possible computer system ")

In [ ]:
print("How many tests did you perform?")
num_tests = int(input())
testnames=[]

# Path for Data Entry form 

The path to the data entry form is the location of the data entry form on your computer
An example of how to get location of a file is 
1) Open File Explorer 
2) Double Click on file (will open a menu of options) 
3) Click copy path 
4) Enter path when prompted but DO NOT INCLUDE QUOATION MARKS (") 

The file input must be a spreadsheet (ends in .xlsx) 

In [ ]:
directory_list=[]
file_list=[]
testname_list = []
for x in range(num_tests):
    sheetinputpath= input("Input path of data entry form (spreadsheet):\n")
    directory, filename = os.path.split(sheetinputpath)
    datadirectory, testname = os.path.split(directory)
    logname = testname + '_log.txt'
    logpath = os.path.join(directory, logname)
    inputpath=sheetinputpath
    outputpath=os.path.join(directory,testname+'_EnergyInputs.csv')
    LEMS_MakeInputFile_EnergyCalcs(inputpath,outputpath,logpath)
    directory_list.append(directory)
    file_list.append(filename)
    testname_list.append(testname)


# Note on Data Files 

All data files must be in the same file directory (same folder) as the excel file given above. From the excel file, the notebook will determine a test name. For future reference, the testname will be considered test in the descriptions below. 


# Energy Metrics 

The Energy Metrics section will calculate values such as initial fuel temperature, initial water temperature, initial mass, final fuel temperature, max water temperature, final mass,useful energy delivered, cooking power, efficiency without char, efficiency with char, burn rate and fire power. These are some of the values calulated at high, medium, and low power. If you conduct multiple tests, there results will be compared in the comparison section. 

Must Have 
1) file in the same folder as the initial excel file with the name "test_EnergyInputs.csv" (in replace of test will be your test name)

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_EnergyOutputs.csv" (in replace of test will be your test name)

In [ ]:
print("Would you like to calculate energy metrics? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1:
    if test_type == 1894: 
            import LEMS_EnergyCalcs as EnergyCalcs
            import LEMS_EnergyCalcs_L2 as EnergyCalcs2  
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else: 
    print("Calculating Energy Metrics skipped")

In [ ]:
if forward == 1:
    if test_type == 1894: 
        for idx in range (num_tests):
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            inputpath=os.path.join(directory_list[idx],testname_list[idx]+'_EnergyInputs.csv')
            outputpath=os.path.join(directory_list[idx],testname_list[idx]+'_EnergyOutputs.csv')
            trial, units, uval = EnergyCalcs.LEMS_EnergyCalcs(inputpath, outputpath, logpath)
            if num_tests == 1: 
                x_data = 0 
                idx = 0
                for key, value in trial['hp'].items(): 
                    x_data = x_data + 1
                data = [0]*x_data
                for key, value in trial['hp'].items():
                    data[idx] = [key, value]
                    idx = idx + 1
                idx = 0
                for key, value in trial['mp'].items():
                    data[idx].append(value)
                    idx = idx + 1
                idx = 0
                for key, value in trial['lp'].items():
                    data[idx].append(value)
                    idx = idx + 1
                idx = 0

                x_const = 20 
                idx = 0
                constants = [0]*x_const
                for key, value in uval.items():
                    if idx < x_const:
                        constants[idx] = [key,value]
                        idx = idx + 1 
        if num_tests > 1: 
            inputpath = []
            for idx in range(num_tests):
                inpath=os.path.join(directory_list[idx],testname_list[idx]+'_EnergyInputs.csv')
                inputpath.append(inpath)

            outputpath=os.path.join(directory,testname+'_EnergyOutputs_Combined.csv')

            trial,averages, data_values, N, stadev, interval, high_tier, low_tier, COV = EnergyCalcs2.LEMS_EnergyCalcs_L2(inputpath, outputpath, testname_list)
            x_data = 0 
            idx = 0
            
            for key, value in trial[testname_list[0]].items(): 
                x_data = x_data + 1
            data = [0]*x_data
            test = 0
            for names in testname_list:
                if test == 0: 
                    for key, value in trial[names].items():
                        data[idx] = [key, value]
                        idx = idx + 1
                    test = test + 1
                    idx= 0
                else:  
                    for key,value in trial[names].items():
                        data[idx].append(value)
                        idx = idx +1
                    test = test + 1
                    idx = 0
    

# Constants 

In [ ]:
if forward == 1:
    if test_type == 1894: 
        if num_tests == 1:
            head = ["Varaible", "Value"]
            print(tabulate(constants, tablefmt="grid"))
        else: 
            print ("no info given")
    

# Engergy Calculations

In [ ]:
if forward == 1:
    if test_type == 1894: 
        if num_tests == 1: 
            head = ["Varaible", "High Power Value", "Medium Power Value","Low Power Value", "Units"]
            print(tabulate(data, headers=head, tablefmt="grid"))
        else: 
            head = [] 
            head.append("Variable")
            for names in testname_list: 
                head.append(names)
            print(tabulate(data, headers=head, tablefmt="grid"))

# Comparison

In [ ]:
if forward == 1: 
    if test_type == 1894: 
        if num_tests == 1:
            print("No comparison needed")
        else:
            x_data = 0
            idx  = 0

            # find number of values 
            for key,value in averages.items():
                x_data = x_data + 1
            holder = [0]*x_data

            # add averages to list 
            for key,value in averages.items():
                holder[idx]=[key,value]
                idx = idx + 1
            idx = 0

            # add N to list 
            for k_N,v_N in N.items():
                holder[idx].append(v_N)
                idx = idx + 1
            idx = 0

            #add stadev to list 
            for k_s,v_s in stadev.items():
                holder[idx].append(v_s)
                idx = idx + 1
            idx = 0

            #add interval to list 
            for k_i,v_i in stadev.items():
                holder[idx].append(v_i)
                idx = idx + 1
            idx = 0

            #add high_tier to list 
            for k_h,v_h in high_tier.items():
                holder[idx].append(v_h)
                idx = idx + 1
            idx = 0

            #add low tier to list 
            for k_l,v_l in low_tier.items():
                holder[idx].append(v_l)
                idx = idx + 1
            idx = 0

            #add COV to list 
            for k_c,v_c in COV.items():
                holder[idx].append(v_c)
                idx = idx + 1
            idx = 0

            head = ["Variable", "Averages", "N","Stadev", "Interval", "High Tier", "Low Tier", "COV"]

            print(tabulate(holder,headers = head, tablefmt="grid"))


# Adjust Calculations

This section will adjust the data due to the sensor calibration. 

A popup window will appear
1) It will ask if you want to save and close the header file
2) Press ok
3) It will ask to enter the sensorbox firmware version
4) Enter the firmware version (SB4003.16 is the default) 
5) Press ok 
6) It will tell you "the following plots show the effect of the recalulation. Close the plots to continue" 
7) Press ok 
8) 2 plots per test will appear in the notebook 

Must Have 
1) file in the same folder as the initial excel file with the name "test_RawData.csv" (in replace of test will be your test name)

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_RawData_Recalibrated.csv" (in replace of test will be your test name)
2) a file in the same folder as the initial excel file with the name "test_Header.csv" (in replace of test will be your test name)

In [ ]:
print("Would you like to adjust sensor calibrations? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1: 
    if test_type == 1894:
        import LEMS_Adjust_Calibrations as adjust
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else:
    print("Adust Sensor Calibrations skipped")

In [ ]:
if forward == 1:  
    if test_type == 1894: 
        for idx in range (num_tests):
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            inputpath=os.path.join(directory_list[idx],testname_list[idx]+'_RawData.csv')
            outputpath=os.path.join(directory_list[idx],testname_list[idx]+'_RawData_Recalibrated.csv')
            headerpath = os.path.join(directory_list[idx],testname_list[idx]+'_Header.csv')
            adjust.LEMS_Adjust_Calibrations(inputpath,outputpath,headerpath,logpath)

# Time Shift Calculations 

This section will complete a time shift to the given data.

A pop up window will appear
1) It will ask you to enter the seconds to shift each data series 
2) Enter the seconds to shift each data series for all the given variables 
3) Press ok (if you can not see ok, grab the top of the popup window and drag until you can see the whole window) 
4) Nothing else will happen and Nothing will appear on the notebook (that is what is supposed to happen) 

Must Have 
1) file in the same folder as the initial excel file with the name "test_RawData_Recalibrated.csv" (in replace of test will be your test name). This will come from running the Adjust Calculation Section 

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section 
3) Adjust Calculations Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_TimeShifts.csv" (in replace of test will be your test name)
2) a file in the same folder as the initial excel file with the name "test_RawData_Shifted.csv" (in replace of test will be your test name)

In [ ]:
print("Would you like to correct for response times? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1: 
    if test_type == 1894:
        import LEMS_ShiftTimeSeries as time
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else: 
    print("Correcting for response times skipped")

In [ ]:
if forward == 1: 
    if test_type == 1894:
        for idx in range(num_tests):
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            inputpath=os.path.join(directory_list[idx],testname_list[idx]+'_RawData_Recalibrated.csv')
            outputpath=os.path.join(directory_list[idx],testname_list[idx]+'_RawData_Shifted.csv')
            timespath = os.path.join(directory_list[idx],testname_list[idx]+'_TimeShifts.csv')
            time.LEMS_ShiftTimeSeries(inputpath,outputpath,timespath,logpath)

#  Subtracting background
This section will take into account the background. 

A popup window will appear 
1) It will ask you to edit phase times in the format hour(hh):minutes(mm):second(ss) where two numbers are given for each category 
2) If the phase times you want are different than what is there (Can skip to step 3) 
    
    2a) edit the phase times 
    
    2b) press ok 
    
    2c) continue this process until all the phase times you want to consider and entered and follow on to step 3 

3) If the phase times you want to subtract are in the box already (the default values) 
    
    3a) Press Cancel 
    
    3b) two graphs and two tables will appear in the notebook 


Must Have 
1) file in the same folder as the initial excel file with the name "test_RawData_Shifted.csv" (in replace of test will be your test name) (comes from Time Shift Calculation Section) 
2) file in the same folder as the initial excel file with the name "test_EnergyInputs.csv" (in replace of test will be your test name)

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section
3) Adjust Calculations Section 
4) Time Shift Calculations Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_TimeSeries.csv" (in replace of test will be your test name)
2) a file in the same folder as the initial excel file with the name "test_Averages.csv" (in replace of test will be your test name)
3) a file in the same folder as the initial excel file with the name "test_PhaseTimes.csv" (in replace of test will be your test name)
4) Graphs and Tables that come after the popup window 

In [ ]:
print("Would you like to subtract background? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1: 
    if test_type == 1894:
        import LEMS_SubtractBkg as sub
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else: 
    print("Subtracting Background skipped")

In [ ]:
if forward == 1: 
    if test_type == 1894: 
        for idx in range(num_tests):   
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            inputpath=os.path.join(directory_list[idx],testname_list[idx]+'_RawData_Shifted.csv')
            energyinputpath = os.path.join(directory_list[idx],testname_list[idx]+'_EnergyInputs.csv')
            outputpath=os.path.join(directory_list[idx],testname_list[idx]+'_TimeSeries.csv')
            aveoutputpath=os.path.join(directory_list[idx],testname_list[idx]+'_Averages.csv')
            timespath = os.path.join(directory_list[idx],testname_list[idx]+'_PhaseTimes.csv')
            sub.LEMS_SubtractBkg(inputpath,energyinputpath,outputpath,aveoutputpath,timespath,logpath)

# Gravimetric Calculations 
This section will calculate Gravimetric Data 


Must Have 
1) file in the same folder as the initial excel file with the name "test_GravInputs.csv" (in replace of test will be your test name)
2) file in the same folder as the initial excel file with the name "test_Averages.csv" (in replace of test will be your test name) (comes from Subtracting Background Section) 

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section
3) Adjust Calculations Section 
4) Time Shifts Calculations Section 
5) Subtracting Background Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_PhaseTimes.csv" (in replace of test will be your test name)
2) a file in the same folder as the initial excel file with the name "test_GravOutputs.csv" (in replace of test will be your test name)
3) a table with the gavimetric data 


In [ ]:
print("Would you like to calculate gravimetric PM? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1: 
    if test_type == 1894: 
        import LEMS_GravCalcs as GravCalcs
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else: 
    print("Gravimetric Calulations skipped")

In [ ]:
if forward == 1:
    if test_type == 1894: 
        for idx in range(num_tests):
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            gravinputpath=os.path.join(directory_list[idx],testname_list[idx]+'_GravInputs.csv')
            aveinputpath = os.path.join(directory_list[idx],testname_list[idx]+'_Averages.csv')
            timespath = os.path.join(directory_list[idx],testname_list[idx]+'_PhaseTimes.csv')
            gravoutputpath=os.path.join(directory_list[idx],testname_list[idx]+'_GravOutputs.csv')
            GravCalcs.LEMS_GravCalcs(gravinputpath,aveinputpath,timespath,gravoutputpath,logpath)

#  Emmision Calculations 
This section will calculate Emissions Data.  

Must Have 
1) file in the same folder as the initial excel file with the name "test_EnergyOutputs.csv" (in replace of test will be your test name)
2) file in the same folder as the initial excel file with the name "test_TimeSeries.csv" (in replace of test will be your test name) (comes from Subtracting Background Section) 
3) file in the same folder as the initial excel file with the name "test_GavOutputs.csv" (in replace of test will be your test name) (comes from Subtracting Background Section)
4) file in the same folder as the initial excel file with the name "test_Averages.csv" (in replace of test will be your test name) (comes from Gavimetric Calculation Section)

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section
3) Energy Calculation Section  
4) Adjust Calculations Section 
5) Time Shifts Calculations Section 
6) Subtracting Background Section 
7) Gavimetric Calculation Section 

Will Output 
1) a file in the same folder as the initial excel file with the name "test_PhaseTimes.csv" (in replace of test will be your test name)
2) a file in the same folder as the initial excel file with the name "test_EmissionOutputs.csv" (in replace of test will be your test name)
3) a file in the same folder as the initial excel file with the name "test_AllOutputs.csv" (in replace of test will be your test name)
4) Data table with emissions data 



In [ ]:
print("Would you like to calculate emission metrics? Press 0-No 1-Yes")
forward = int(input())

In [ ]:
if forward == 1: 
    if test_type == 1894: 
        import LEMS_EmissionCalcs as emis
    elif test_type == 1952:
        print("No code has yet been figured out for this test type")
    else: 
        print("Test Type unavaible")
else: 
    print("Calulating Emission Metrics skipped")

In [ ]:
if forward == 1: 
    if test_type == 1894: 
        for idx in range(num_tests):
            cstr= " This is for " + testname_list[idx] + " "
            print (Fore.BLUE + cstr.center(70, '-'))
            print(Style.RESET_ALL)
            inputpath=os.path.join(directory_list[idx],testname_list[idx]+'_TimeSeries.csv')
            energypath=os.path.join(directory_list[idx],testname_list[idx]+'_EnergyOutputs.csv')
            gravinputpath=os.path.join(directory_list[idx],testname_list[idx]+'_GravOutputs.csv')
            aveinputpath = os.path.join(directory_list[idx],testname_list[idx]+'_Averages.csv')
            timespath = os.path.join(directory_list[idx],testname_list[idx]+'_PhaseTimes.csv')
            emisoutputpath=os.path.join(directory_list[idx],testname_list[idx]+'_EmissionOutputs.csv')
            alloutputpath=os.path.join(directory_list[idx],testname_list[idx]+'_AllOutputs.csv')
            allnames,allunits,allval,allunc,alluval = emis.LEMS_EmissionCalcs(inputpath,energypath,gravinputpath,aveinputpath,emisoutputpath,alloutputpath,logpath)
            data = [0]*len(allnames)
            for x in range(len(allnames)):
                data[x] = [allnames[x], allunits[allnames[x]], allval[allnames[x]], allunc[allnames[x]]]
            print(tabulate(data, tablefmt="grid"))


#  Real Time Data Plotting 
This section will plot the real time data. 

Must Have 
1) file in the same folder as the initial excel file with the name "test_TimeSeries.csv" (in replace of test will be your test name) (comes from Subtracting Background Section) 

Must Run 
1) SetUp Section 
2) Path for Data Entry Form Section
3) Energy Calculation Section  
4) Adjust Calculations Section 
5) Time Shifts Calculations Section 
6) Subtracting Background Section 


Will Output 
1) a file in the same folder as the initial excel file with the name "test_plots.csv" (in replace of test will be your test name)
2) a graph 

You can update the test_plots.csv file to change your plot. If you make a change to the tests_plots.csv file you will need to rerun the plotting code 

To rerun the plotting code 
1) Scoll to the top of the notebook and click "here" to show the code 
2) Scoll back down to the bottom of the code and select the box below this (it will say import PEMS_Plotter1 as plotter at the top 
3) Hit run at the top of the screen (there is a sideways triangle before the word run) 
4) Scoll back to the top of the notebook and click "here" to hide the code 


In [ ]:
import PEMS_Plotter1 as plotter
inputpath = os.path.join(directory_list[0],testname_list[0]+'_TimeSeries.csv')
fuelpath = "hello"
exactpath = "hello"
plotpath = os.path.join(directory_list[0],testname_list[0]+'_plots.csv')
plotter.PEMS_Plotter(inputpath, fuelpath, exactpath, plotpath)